In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from scipy.io import loadmat
from scipy.stats import norm

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import hsv_to_rgb
from matplotlib.patches import Circle

In [ ]:
from bayesee.evaluation import *

In [ ]:
%load_ext autoreload
%autoreload 2
plt.style.use('bayesee.academic')

In [ ]:
repo_path = Path.cwd().parents[0]
print(repo_path)

In [ ]:
subject = "AZ_t1"
file_name = repo_path / f"data/covert-search/large-field/p2_data_{subject}.mat"
data = loadmat(file_name)
print(data.keys())

In [ ]:
target_amplitude = data["targetAmplitude"]
target_location = data["tLocation"]
spot_center = data["spotCenters"]
human_response = data["hResponse"]
n_location = data["nLocations"][0][0]
spot_diameter = data["spotLength"][0][0]
stimulus_design_size = data["totalLength"][0][0]
monitor_width = data["monitorPx"][0][0]
monitor_height = data["monitorPx"][0][1]
ppd = data["ppd"][0][0]

In [ ]:
shifted_spot_center = spot_center.copy()
shifted_spot_center[:, 0] += (monitor_height - stimulus_design_size) // 2
shifted_spot_center[:, 1] += (monitor_width - stimulus_design_size) // 2

accurate_response = target_location == human_response

In [ ]:
list_spot_region = [np.zeros(monitor_width, monitor_height)] * (n_location - 1)
pixel_col, pixel_row = np.meshgrid(np.arange(monitor_width), np.arange(monitor_height))

for index_location in range(n_location - 1):
    list_spot_region[index_location] = (
        pixel_row - shifted_spot_center[index_location, 0]
    ) ** 2 + (
        pixel_col - shifted_spot_center[index_location, 1]
    ) ** 2 <= spot_diameter**2 / 4

In [ ]:
stimulus_region = np.zeros((monitor_height, monitor_width))

for index_location in range(n_location - 1):
    stimulus_region[list_spot_region[index_location]] = 1

In [ ]:
fig, ax = plt.subplots()
ax.imshow(stimulus_region)

for index_location in range(n_location - 1):
    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{index_location+1}",
        ha="center",
        va="center",
    )

In [ ]:
local_accuracy_region = np.empty((monitor_height, monitor_width))

local_accuracy = accurate_response.mean(axis=(0, 1, 2))

for index_location in range(n_location - 1):
    local_accuracy_region[list_spot_region[index_location]] = local_accuracy[
        index_location
    ]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_accuracy_region)

for index_location in range(n_location - 1):
    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{local_accuracy[index_location]:.3f}",
        ha="center",
        va="center",
    )

In [ ]:
local_hit_region = np.empty((monitor_height, monitor_width))

local_hit = np.array(
    [
        np.logical_and(
            target_location[:, :, :, index_location] == index_location + 1,
            human_response[:, :, :, index_location] == index_location + 1,
        ).sum()
        / (target_location[:, :, :, index_location] == index_location + 1).sum()
        for index_location in range(n_location - 1)
    ]
)

for index_location in range(n_location - 1):
    local_hit_region[list_spot_region[index_location]] = local_hit[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_hit_region)

for index_location in range(n_location - 1):
    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{local_hit[index_location]:.3f}",
        ha="center",
        va="center",
    )

In [ ]:
local_cr_region = np.empty((monitor_height, monitor_width))

local_cr = np.array(
    [
        np.logical_and(
            target_location[:, :, :, index_location] == 0,
            human_response[:, :, :, index_location] == 0,
        ).sum()
        / (target_location[:, :, :, index_location] == 0).sum()
        for index_location in range(n_location - 1)
    ]
)

for index_location in range(n_location - 1):
    local_cr_region[list_spot_region[index_location]] = local_cr[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_cr_region)

for index_location in range(n_location - 1):
    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{local_cr[index_location]:.2f}",
        ha="center",
        va="center",
    )

In [ ]:
local_dp_region = np.empty((monitor_height, monitor_width))

## Assumption: if we were to double the number of trials, one error would be made
tol = 1e-5
local_hit_for_dp = local_hit.copy()
local_cr_for_dp = local_cr.copy()
n_trials_by_location = target_location.size / target_location.shape[-1]
assumed_local_accuracy = (n_trials_by_location * 2 - 1) / (n_trials_by_location * 2)

for index_location in range(n_location - 1):
    if abs(local_hit[index_location] - 1.0) < tol:
        local_hit_for_dp[index_location] = assumed_local_accuracy

    if abs(local_cr[index_location] - 1.0) < tol:
        local_cr_for_dp[index_location] = assumed_local_accuracy


local_dp = norm.ppf(local_hit_for_dp) - norm.ppf(1 - local_cr_for_dp)

for index_location in range(n_location - 1):
    local_dp_region[list_spot_region[index_location]] = local_dp[index_location]

np.save(
    repo_path / f"data/covert-search/large-field/derived/est_dp_{subject}", local_dp
)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_dp_region)

for index_location in range(n_location - 1):
    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{local_dp[index_location]:.3f}",
        ha="center",
        va="center",
    )

In [ ]:
local_pc_max_region = np.empty((monitor_height, monitor_width))

local_pc_max = norm.cdf(local_dp / 2)

for index_location in range(n_location - 1):
    local_pc_max_region[list_spot_region[index_location]] = local_pc_max[index_location]

In [ ]:
fig, ax = plt.subplots()
ax.imshow(local_pc_max_region)

for index_location in range(n_location - 1):
    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{local_pc_max[index_location]:.3f}",
        ha="center",
        va="center",
    )

In [ ]:
array_eccentral_distance = np.zeros((n_location - 1,))

for index_location in range(n_location - 1):
    array_eccentral_distance[index_location] = np.sqrt(
        (spot_center[index_location, 0] - stimulus_design_size // 2) ** 2
        + (spot_center[index_location, 1] - stimulus_design_size // 2) ** 2
    )

array_eccentral_distance /= ppd

print(array_eccentral_distance)

In [ ]:
pixel_precision_array_eccentral_distance = array_eccentral_distance.copy()

for index_d1, distance1 in enumerate(np.unique(array_eccentral_distance)):
    for index_d2, distance2 in enumerate(
        np.unique(array_eccentral_distance)[index_d1 + 1 :]
    ):
        if distance1 != distance2 and np.abs(distance2 - distance1) < 0.5:
            print(index_d1, distance1, index_d2, distance2)
            pixel_precision_array_eccentral_distance[
                array_eccentral_distance == distance2
            ] = distance1

print(pixel_precision_array_eccentral_distance)

In [ ]:
orientation_hsv_color_map = np.zeros((n_location - 1, 3))

for index_location in range(n_location - 1):
    dy = spot_center[index_location, 0] - stimulus_design_size // 2
    dx = stimulus_design_size // 2 - spot_center[index_location, 1]
    orientation_hsv_color_map[index_location, 0] = (1 - np.arctan2(dy, dx) / np.pi) / 2

    orientation_hsv_color_map[index_location, 1] = 1

    if dx == 0 and dy == 0:
        orientation_hsv_color_map[index_location, 2] = 0
    else:
        orientation_hsv_color_map[index_location, 2] = 0.75

rgb_orientation_color_map = hsv_to_rgb(orientation_hsv_color_map)

In [ ]:
fig, ax = plt.subplots()
stimulus_region = np.zeros((monitor_height, monitor_width))
ax.imshow(stimulus_region)

for index_location in range(n_location - 1):
    circle = Circle(
        (
            shifted_spot_center[index_location, 1],
            shifted_spot_center[index_location, 0],
        ),
        spot_diameter / 2,
        color=rgb_orientation_color_map[index_location],
        linewidth=1,
    )
    ax.add_patch(circle)

    ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{index_location+1}",
        ha="center",
        va="center",
        c="w",
    )

In [ ]:
fig, ax = plt.subplots()
ax.scatter(
    pixel_precision_array_eccentral_distance,
    local_accuracy,
    s=250,
    c=rgb_orientation_color_map,
)

ax.set(xlabel="Eccentral distance (deg)", ylabel="Accuracy")

inner_ax = fig.add_axes([0.2, 0.1, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_location in range(n_location - 1):
    circle = Circle(
        (
            shifted_spot_center[index_location, 1],
            shifted_spot_center[index_location, 0],
        ),
        spot_diameter / 2,
        color=rgb_orientation_color_map[index_location],
        linewidth=1,
    )
    inner_ax.add_patch(circle)

    inner_ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{index_location+1}",
        ha="center",
        va="center",
        c="w",
        fontsize=16,
    )

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(array_eccentral_distance, local_hit, s=250, c=rgb_orientation_color_map)

ax.set(xlabel="Eccentral distance (deg)", ylabel="Hit")

inner_ax = fig.add_axes([0.2, 0.1, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_location in range(n_location - 1):
    circle = Circle(
        (
            shifted_spot_center[index_location, 1],
            shifted_spot_center[index_location, 0],
        ),
        spot_diameter / 2,
        color=rgb_orientation_color_map[index_location],
        linewidth=1,
    )
    inner_ax.add_patch(circle)

    inner_ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{index_location+1}",
        ha="center",
        va="center",
        c="w",
        fontsize=16,
    )

plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.scatter(array_eccentral_distance, local_cr, s=250, c=rgb_orientation_color_map)

ax.set(xlabel="Eccentral distance (deg)", ylabel="Correct rejection")

inner_ax = fig.add_axes([0.2, 0.1, 0.288, 0.461])
inner_ax.imshow(stimulus_region)
inner_ax.set(xticks=[], yticks=[])

for index_location in range(n_location - 1):
    circle = Circle(
        (
            shifted_spot_center[index_location, 1],
            shifted_spot_center[index_location, 0],
        ),
        spot_diameter / 2,
        color=rgb_orientation_color_map[index_location],
        linewidth=1,
    )
    inner_ax.add_patch(circle)

    inner_ax.text(
        shifted_spot_center[index_location, 1],
        shifted_spot_center[index_location, 0],
        f"{index_location+1}",
        ha="center",
        va="center",
        c="w",
        fontsize=16,
    )

plt.show()

In [ ]:
local_accuracy_by_session = accurate_response.mean(axis=(0, 2))
n_session = local_accuracy_by_session.shape[0]
array_index_location = np.arange(1, n_location)

In [ ]:
fig, ax = plt.subplots()

for index_session in range(n_session):
    ax.scatter(
        array_index_location,
        local_accuracy_by_session[index_session, :],
        s=250,
        label=f"Session {index_session+1}",
    )

ax.legend(loc="best")
ax.set(xlabel="Spot", ylabel="Accuracy", xticks=array_index_location)
plt.show()